# About
We are going to build a recurrent neural network that learns to classify if the sentence is solely composed of even or odd digits. Although deep learning is unncessary for the task, this simple practice helps to understand the basics of RNN. So, let's build it!

In [1]:
import numpy as np
import tensorflow as tf

/Users/shatapy/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
batch_size = 128
h_size = 128
embedding_dim = 64
n_class = 2
time_steps = 6
element_size = 1

In [3]:
num_to_word_map = {1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', 6: 'six', 7: 'seven', 8: 'eight', 9: 'nine'}
num_to_word_map[0] = 'PAD'  # zero padding for the sentence shorter than time_steps 6

In [4]:
even_sent = []  # list of sentences composed of even digits
odd_sent = []
seq_lens = []  # list of lengths of sequences

### Generate odd and even sentences

We put a zero padding to the sentences shorter than `time_steps 6` to feed data of consistent shape to basic rnn network. However, this creates an unnecessary noise that the network has to learn additionally. So, make sure tolet `tf.nn.dynamic_rnn()` know where the sentence ends so that it can only consider the necessary features.

In [5]:
for i in range(10000):
    rand_seq_len = np.random.randint(3, 7)  # length of 3 ~ 6
    seq_lens.append(rand_seq_len)
    
    odd_nums = np.random.choice(range(1, 10, 2), rand_seq_len)  # 3, 5, 1, 3
    even_nums = np.random.choice(range(2, 10, 2), rand_seq_len)
    
    if rand_seq_len < 6:
        odd_nums = np.append(odd_nums, [0]*(6-rand_seq_len))
        even_nums = np.append(even_nums, [0]*(6-rand_seq_len))
    
    odd_sent.append(" ".join([num_to_word_map[i] for i in odd_nums]))
    even_sent.append(" ".join([num_to_word_map[i] for i in even_nums]))
    
data = even_sent + odd_sent  # remember this order. even first. odd later!
seq_lens *= 2

##### Test

In [6]:
even_sent[0:10]

['two four six eight PAD PAD',
 'two six two six eight PAD',
 'two eight six PAD PAD PAD',
 'eight eight two PAD PAD PAD',
 'four eight eight four two four',
 'two eight eight PAD PAD PAD',
 'four six six PAD PAD PAD',
 'eight four two eight PAD PAD',
 'six two four six PAD PAD',
 'two six two two four two']

## Map the "Token Ids" to each digits
The ids don't have a special meaning in itself. They are just indices randomly put to each digits they meet first. <strong>This means that the id 1 doesn't necessary imply that its value is one.

In [7]:
word2index = {}
idx = 0
for sent in data:
    for word in sent.split():
        if word not in word2index:
            word2index[word] = idx
            idx += 1

index2word = {index:word for word, index in word2index.items()}

In [8]:
vocab_size = len(word2index)

In [9]:
index2word

{0: 'two',
 1: 'four',
 2: 'six',
 3: 'eight',
 4: 'PAD',
 5: 'one',
 6: 'three',
 7: 'five',
 8: 'seven',
 9: 'nine'}

## Make one-hot labels and split data into train and test datasets

In [10]:
labels = [1]*10000 + [0]*10000 # [even*10000, odd*10000]. Remember the order of data above? Even first. Odd later

for i in range(len(labels)):
    label = labels[i]
    one_hot = [0, 0]
    one_hot[label] = 1  # if label == 1, then one_hot=[0, 1], which is even.
    labels[i] = one_hot
    
indices = list(range(len(data)))
np.random.shuffle(indices)
    
data = np.array(data)[indices]
labels = np.array(labels)[indices]
seq_lens = np.array(seq_lens)[indices]

num_test = len(data) // 10  # train: test = 9: 1

train_x = data[num_test:]
train_y = labels[num_test:]
train_seq_lens = seq_lens[num_test:]

test_x = data[:num_test]
test_y = labels[:num_test]
test_seq_lens = seq_lens[:num_test]

In [11]:
def get_sent_batch(batch_size, data_x, data_y, data_seq_lens):
    temp_indices = list(range(len(data_x)))
    np.random.shuffle(temp_indices)
    batch_indices = temp_indices[:batch_size]
    x = [[word2index[word] for word in data_x[i].split()] for i in batch_indices]
    y = [data_y[i] for i in batch_indices]
    seqlens = [data_seq_lens[i] for i in batch_indices]
    return x, y, seqlens

In [12]:
X = tf.placeholder(tf.int32, [batch_size, time_steps], name='X')
Y = tf.placeholder(tf.float32, [batch_size, n_class], name='Y')

_seqlens = tf.placeholder(tf.int32, [batch_size])

## Embed the words to vectors

In [13]:
with tf.name_scope('embeddings'):
    embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_dim], -1., 1., name='embedding'))
    embed = tf.nn.embedding_lookup(embeddings, X)

In [14]:
with tf.variable_scope('LSTM'):
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(h_size, forget_bias=1.)
    outputs, states = tf.nn.dynamic_rnn(lstm_cell, embed, sequence_length=_seqlens, dtype=tf.float32)
    W = tf.get_variable('W_linear', [h_size, n_class], initializer=tf.contrib.layers.xavier_initializer())
    b = tf.get_variable('b_linear', [n_class], initializer=tf.zeros_initializer())
    
    logits = tf.matmul(states[1], W) + b
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))

In [15]:
train_op = tf.train.AdamOptimizer().minimize(loss)

In [16]:
correct_preds = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
acc_op = tf.reduce_mean(tf.cast(correct_preds, tf.float32)) * 100

In [27]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(1000):
        x_batch, y_batch, seqlen_batch = get_sent_batch(batch_size, train_x, train_y, train_seq_lens)
        sess.run(train_op, feed_dict={X: x_batch, Y: y_batch, _seqlens: seqlen_batch})
        
        if not epoch % 100:
            _loss, acc = sess.run([loss, acc_op], feed_dict={X: x_batch, Y: y_batch, _seqlens: seqlen_batch})
            print('Epoch: ', epoch, ' Loss: ', _loss, ' Acc: ', acc)
    
    for i in range(5):
        x_test, y_test, seqlen_test = get_sent_batch(batch_size, test_x, test_y, test_seq_lens)
        pred, test_acc = sess.run([tf.argmax(logits, 1), acc_op], feed_dict={X: x_test, Y: y_test, _seqlens: seqlen_test})
        print('Acc: ', acc)
    
    output_ex, state_ex = sess.run([outputs, states], feed_dict={X: x_test, Y: y_test, _seqlens: seqlen_test})
    print('output: ', output_ex)
    print('state: ', state_ex)

Epoch:  0  Loss:  0.6158763  Acc:  85.9375
Epoch:  100  Loss:  0.00043691305  Acc:  100.0
Epoch:  200  Loss:  0.00015363062  Acc:  100.0
Epoch:  300  Loss:  9.8566146e-05  Acc:  100.0
Epoch:  400  Loss:  5.151487e-05  Acc:  100.0
Epoch:  500  Loss:  4.837307e-05  Acc:  100.0
Epoch:  600  Loss:  2.89299e-05  Acc:  100.0
Epoch:  700  Loss:  2.164101e-05  Acc:  100.0
Epoch:  800  Loss:  1.5136298e-05  Acc:  100.0
Epoch:  900  Loss:  1.0729499e-05  Acc:  100.0
Acc:  100.0
Acc:  100.0
Acc:  100.0
Acc:  100.0
Acc:  100.0
output:  [[[ 0.06128662  0.04333561 -0.05865165 ...  0.09651218  0.05022084
   -0.05134832]
  [ 0.08676399 -0.07055885 -0.10745798 ...  0.4698604   0.35953382
   -0.44778892]
  [ 0.18908355 -0.11490053 -0.23027565 ...  0.67016816  0.49687293
   -0.63750213]
  [ 0.41138342 -0.32771516 -0.23151074 ...  0.88121563  0.7714114
   -0.8990004 ]
  [ 0.3105936  -0.5110374  -0.21220931 ...  0.9636515   0.8867327
   -0.9711413 ]
  [ 0.          0.          0.         ...  0.          0